In [1]:
!mkdir corpus

In [1]:
!pip install pyserini
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyserini-0.21.0-py3-none-any.whl (154.1 MB)
  Using cached pyjnius-1.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached nmslib-2.1.1.tar.gz (188 kB)
  Preparing metadata (setup.py) ... done
  Using cached sentencepiece-0.1.98-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached onnxruntime-1.14.1-cp310-cp310-manylinux_2_27_x86_64.whl (5.0 MB)
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-l

In [3]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/drive/MyDrive/BM25ODQA/corpus \
  --language zh \
  --index /content/drive/MyDrive/BM25ODQA/indexes/wiki \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2023-04-30 13:17:36,946 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-30 13:17:36,949 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-30 13:17:36,949 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-30 13:17:36,949 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/drive/MyDrive/BM25ODQA/corpus
2023-04-30 13:17:36,950 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-30 13:17:36,950 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-30 13:17:36,951 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2023-04-30 13:17:36,951 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: zh
2023-04-30 13:17:36,952 INFO  [main] index.

In [4]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
def QAmodel(question,paragraph):
    encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)

    inputs = encoding['input_ids']  #Token embeddings
    sentence_embedding = encoding['token_type_ids']  #Segment embeddings
    tokens = tokenizer.convert_ids_to_tokens(inputs)
    start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
    start_index = torch.argmax(start_scores)

    end_index = torch.argmax(end_scores)

    answer = ' '.join(tokens[start_index:end_index+1])
    corrected_answer = ''

    for word in answer.split():
        
        #If it's a subword token
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word

    return corrected_answer

In [11]:
import json
f=open("/content/drive/MyDrive/BM25ODQA/corpus/inv_wiki_tra.json","r",encoding='UTF-8')
print(f.read()[0])


UnicodeDecodeError: ignored

In [ ]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher.from_prebuilt_index('/content/drive/MyDrive/BM25ODQA/indexes/wiki')
question="王建民事誰"
hits = searcher.search(question)
def search_paragragh(id):
    for i in data:
        if i["id"]==id:
            return i["contents"]
print(QAmodel(question,search_paragragh(hits[0].docid)))